In [3]:
import random
import numpy as np
import timeit
import matplotlib.pyplot as plt
from citipy import citipy
import requests
from config import weather_api_key
from datetime import datetime
import pandas as pd
import time
from scipy.stats import linregress

In [4]:
# Create a set of 2000 random lats x longs
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)
coordinates = list(lat_lngs)
# Peek at the first few coordinates
coordinates[0:5]

[(52.0851538066259, 73.38745210514767),
 (-65.86258799427036, -26.573373554436245),
 (52.989968693181595, 87.67382649364816),
 (-84.27768401629469, -70.89534831657213),
 (-56.18405475928092, 147.15298449138004)]

In [5]:
# Create a list of cities nearest to our 2000 coordinates
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    if city not in cities:
        cities.append(city)

# Print the city count to check count.
len(cities)

625

In [6]:
# Create base URL for making API calls to OWM
APIurl = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [7]:
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval...  ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = APIurl + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_description = city_weather["weather"][0]["description"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Weather Description": city_description,
                          "Date": city_date})

# If an error is experienced, skip the city.
    except:
        print(f"City {city} not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete!")

 Set 4 | natal
Processing Record 33 of Set 4 | longyearbyen
Processing Record 34 of Set 4 | kruisfontein
Processing Record 35 of Set 4 | kodiak
Processing Record 36 of Set 4 | nhulunbuy
Processing Record 37 of Set 4 | hami
Processing Record 38 of Set 4 | attawapiskat
City attawapiskat not found. Skipping...
Processing Record 39 of Set 4 | dong hoi
Processing Record 40 of Set 4 | matagami
Processing Record 41 of Set 4 | lompoc
Processing Record 42 of Set 4 | muisne
Processing Record 43 of Set 4 | tuatapere
Processing Record 44 of Set 4 | marica
Processing Record 45 of Set 4 | rio grande
Processing Record 46 of Set 4 | martapura
Processing Record 47 of Set 4 | januaria
Processing Record 48 of Set 4 | broome
Processing Record 49 of Set 4 | sao francisco
Processing Record 50 of Set 4 | am timan
Processing Record 1 of Set 5 | thinadhoo
Processing Record 2 of Set 5 | salalah
Processing Record 3 of Set 5 | santa maria
Processing Record 4 of Set 5 | gornyy
Processing Record 5 of Set 5 | gamba


In [8]:
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Weather Description,Date
0,Bestobe,52.4994,73.0975,7.63,95,100,8.25,KZ,overcast clouds,2021-01-31 23:53:49
1,Mar Del Plata,-38.0023,-57.5575,64.40,68,75,12.66,AR,broken clouds,2021-01-31 23:41:30
2,Ushuaia,-54.8000,-68.3000,53.60,62,90,8.05,AR,overcast clouds,2021-01-31 23:43:56
3,Hobart,-42.8794,147.3294,75.00,56,75,11.50,AU,broken clouds,2021-01-31 23:44:09
4,Kapaa,22.0752,-159.3190,77.00,69,40,12.66,US,scattered clouds,2021-01-31 23:43:54
5,Georgetown,5.4112,100.3354,78.80,73,20,2.30,MY,few clouds,2021-01-31 23:44:03
6,Beloha,-25.1667,45.0500,75.74,83,51,1.97,MG,broken clouds,2021-01-31 23:51:43
7,Zhireken,52.8249,117.3076,-30.82,81,83,2.95,RU,broken clouds,2021-01-31 23:53:50
8,Avarua,-21.2078,-159.7750,78.80,89,90,3.44,CK,light rain,2021-01-31 23:44:00
9,Severo-Kurilsk,50.6789,156.1250,22.84,91,92,39.55,RU,light snow,2021-01-31 23:43:33


In [9]:
# Create the output file name
output_data_file = "WeatherPy_Database.csv"
# Export the data into output csv 
city_data_df.to_csv(output_data_file, index_label="City_ID")